## Underfitting y Overfitting

Esta notebook tiene como objetivo permitir adquirir intuiciones sobre las nociones de underfitting y overfitting.
En la misma se asumen que una variables $y_i$ efectivamente es generada por $f(x_i) + \epsilon_i$. Siendo $f(x) = sin(2 * \pi * x_i)$.

Teniendo esto en cuenta se genera un conjunto de entrenamiento y otro de testeo. Luego se permite al usuario ajustar interactivamente regresiones polinómicas, pudiendo variar el grado de la regresión y el tamaño del conjunto de entrenamiento.


In [1]:
from ipywidgets import interact
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from IPython.display import clear_output

# Constantes globales
SEED = 1019
MAX_DEGREE = 12

def f(x):
    """Función subyacente verdadera (onda senoidal)."""
    return np.sin(2 * math.pi * x)

def gen_data(size, noise_level, seed=None):
    """
    Genera datos sintéticos.

    Parámetros:
    - size (int): Número de puntos de datos a generar.
    - noise_level (float): Cantidad de ruido para agregar a los datos.
    - seed (int, opcional): Semilla aleatoria para reproducibilidad.

    Devuelve:
    - df (pd.DataFrame): Datos generados en un DataFrame con columnas "x" e "y".
    """
    np.random.seed(seed)
    x = np.random.uniform(size=size)

    # Genera valores de y usando la función verdadera y ruido
    np.random.seed(seed)
    y = f(x) + noise_level * np.random.normal(scale=noise_level, size=size)

    df = pd.DataFrame({"x": x, "y": y})
    return df

def poly_reg_preds(df, x_to_pred, degree):
    """
    Realiza predicciones de regresión polinómica.

    Parámetros:
    - df (pd.DataFrame): Datos de entrenamiento con columnas "x" e "y".
    - x_to_pred (array-like): Valores para predecir "y".
    - degree (int): Grado del polinomio de regresión.

    Devuelve:
    - preds (pd.DataFrame): Valores predichos en un DataFrame con columnas "x" e "y".
    """
    x = df["x"].values.reshape(-1, 1)
    poly = PolynomialFeatures(degree)
    X = poly.fit_transform(x)
    poly_reg = LinearRegression(fit_intercept=False).fit(X, df["y"])

    preds = pd.DataFrame()
    preds["x"] = x_to_pred
    preds["y"] = poly_reg.predict(poly.fit_transform(preds["x"].values.reshape(-1, 1)))

    return preds

def plot_data_and_predictions(ax, df_train, degree):
    """
    Grafica los datos de entrenamiento, la función verdadera y la función predicha.

    Parámetros:
    - ax (plt.Axes): Ejes de Matplotlib en los que se realizará la trama.
    - df_train (pd.DataFrame): Datos de entrenamiento con columnas "x" e "y".
    - degree (int): Grado del polinomio de regresión.

    Devuelve:
    - y_train_preds (array-like): Valores y predichos para los datos de entrenamiento.
    """
    # Grafica los datos de entrenamiento
    ax.scatter(df_train["x"], df_train["y"], s=60, alpha=0.3, color="blue")
    y_train_preds = poly_reg_preds(df_train, df_train["x"], degree)["y"]

    # Grafica la función verdadera
    x = np.linspace(0, 1, 1000)
    ax.plot(x, f(x), color="green", linestyle="-")

    # Grafica la función predicha
    preds = poly_reg_preds(df_train, x, degree)
    ax.plot(preds["x"], preds["y"], color="red", linestyle="--")

    # Define los límites de x e y
    ax.set_ylim([-2.5, 2.5])
    ax.set_xlim([0, 1])

    # Etiquetas de los ejes
    ax.set_xlabel('x')
    ax.set_ylabel('y')

    return y_train_preds

def plot_perf_by_degree(ax, df_train, df_test, degree, max_degree):
    """
    Grafica el rendimiento del modelo en función del grado del polinomio.

    Parámetros:
    - ax (plt.Axes): Ejes de Matplotlib en los que se realizará la trama.
    - df_train (pd.DataFrame): Datos de entrenamiento con columnas "x" e "y".
    - df_test (pd.DataFrame): Datos de prueba con columnas "x" e "y".
    - degree (int): Grado actual del polinomio.
    - max_degree (int): Grado máximo del polinomio.

    No devuelve nada; la función genera la trama directamente.
    """
    rmse = {}
    max_rmse = 0
    for d in range(0, max_degree + 1):
        y_train_preds = poly_reg_preds(df_train, df_train["x"], d)["y"]
        y_test_preds = poly_reg_preds(df_train, df_test["x"], d)["y"]
        rmse_train = math.sqrt(mean_squared_error(df_train["y"], y_train_preds))
        rmse_test = math.sqrt(mean_squared_error(df_test["y"], y_test_preds))
        if max(rmse_train, rmse_test) > max_rmse:
            max_rmse = max(rmse_train, rmse_test)
        rmse[d] = {"rmse_train": rmse_train,
                   "rmse_test": rmse_test}

    # Grafica las curvas de RMSE para entrenamiento y prueba
    ax.plot(range(0, max_degree + 1),
            [rmse[e]["rmse_train"] for e in rmse],
            color="green", linestyle="-", label="train")

    ax.plot(range(0, max_degree + 1),
            [rmse[e]["rmse_test"] for e in rmse],
            color="red", linestyle="--", label="test")

    ax.scatter(2 * [degree],
               [rmse[degree]["rmse_train"], rmse[degree]["rmse_test"]],
               color="blue", linestyle="-")

    # Define los límites de x e y
    ax.set_ylim([0, max_rmse])
    ax.set_xlim([0, max_degree])

    # Etiquetas de los ejes
    ax.set_xlabel('degree')
    ax.set_ylabel('rmse')

    plt.legend()

def update(train_size=40, degree=0, noise_level=0.7):
    """
    Actualiza la trama interactiva en función de la entrada del usuario.

    Parámetros:
    - train_size (int): Número de puntos de datos de entrenamiento.
    - degree (int): Grado del polinomio de regresión.
    """
    fig, ax = plt.subplots(1, 2, figsize=(8, 4))

    # Genera datos de entrenamiento y prueba
    df_train = gen_data(size=train_size, noise_level=noise_level, seed=SEED)
    df_test = gen_data(size=100, noise_level=noise_level, seed=SEED + 1)

    # Grafica los datos de entrenamiento y la función predicha
    y_train_preds = plot_data_and_predictions(ax[0], df_train, degree)

    plot_perf_by_degree(ax[1], df_train, df_test, degree, MAX_DEGREE)

    # Obtiene predicciones de prueba
    y_test_preds = poly_reg_preds(df_train, df_test["x"], degree)["y"]

    # Limpia la salida
    fig.canvas.draw()

    # Calcula y muestra RMSE para datos de entrenamiento y prueba
    train_rmse = math.sqrt(mean_squared_error(df_train["y"], y_train_preds))
    test_rmse = math.sqrt(mean_squared_error(df_test["y"], y_test_preds))

    print(f"Train RMSE: {round(train_rmse, 4)}")
    print(f"Test RMSE: {round(test_rmse, 4)}")

# Crea un widget interactivo
interact(update, train_size=(1, 2000, 1), degree=(0, MAX_DEGREE, 1), noise_level=(0, 2, 0.1));


interactive(children=(IntSlider(value=40, description='train_size', max=2000, min=1), IntSlider(value=0, descr…